https://tushare.pro/document

In [1]:
# %pip install tushare --upgrade

In [2]:
pwd

'/Users/winston/git_rep/sys23/data'

In [3]:
_PATH = '/Users/winston/git_rep/sys23/data/'

import yaml
conf = '/Users/winston/git_rep/sys23/config_stk.yaml'
conf = yaml.safe_load(open(conf, encoding='utf-8'))

In [4]:
import os
import pandas as pd
import numpy as np
import datetime
import time
import importlib
from tqdm import tqdm

In [5]:
import sys
# sys.path.append(conf['api_path'])
sys.path.append('/Users/winston/git_rep/sys23')
import data.api
importlib.reload(data.api)
# from plt_head import *

import tushare as ts
_TOKEN = open(conf['tushare_token']).read().strip('\n')
pro = ts.pro_api(_TOKEN)
# ts.set_token(_TOKEN)
# pro = ts.pro_api()

### 获取交易日期

In [6]:
## 获取交易日期
start_date = '20150101'  # tushare最开始从2015
end_date = datetime.date.today().strftime('%Y%m%d')
# end_date = '20230130'
# calendar_dates = data.api.wget_tradedates(start_date, end_date, pro, path=_PATH)
tradedates = data.api.get_tradedates(start_date, end_date, pro, path=_PATH, fmt='%Y%m%d')

Update `/Users/winston/git_rep/sys23/data//tradedates.csv`
get_tradedates: 20150105 20230210


### 获取日线行情 

In [7]:
## 获取日线行情 

# df = pro.daily(trade_date=tradedates.iloc[0])

api_kws = ['daily', 'daily_basic', 'adj_factor', 
           'suspend_d', 'moneyflow', 'stk_limit', 
           # 'index_dailybasic', # 大盘指数每日指标
           # moneyflow_hsgt, 
          ]

for api_kw in api_kws:
    cache_path = _PATH + f'cache/{api_kw}/'
    os.makedirs(cache_path, exist_ok=True)
    
    bar = tqdm(tradedates)
    for td in bar:
        file = cache_path + td + '.csv'
        if os.path.exists(file):
            continue
        # break
        for _ in range(3):  # 尝试3次
            try:
                df = pro.query(api_kw, trade_date=td)
            except:
                time.sleep(1)
            else:
                df.to_csv(file, index=False)
                break  # 找到并存储
        bar.set_description(api_kw + ': ' + td)

df = pd.read_csv(cache_path + tradedates.iloc[-1] + '.csv')
df.tail()

stk_limit: 20230210: 100%|█████████████████| 1971/1971 [00:05<00:00, 333.61it/s]


,trade_date,ts_code,up_limit,down_limit
6386,20230210,900947.SH,0.29,0.24
6387,20230210,900948.SH,1.54,1.26
6388,20230210,900952.SH,0.31,0.25
6389,20230210,900953.SH,0.51,0.42
6390,20230210,900957.SH,0.63,0.52
